<center> <b>UCGIS I-GUIDE Community Champion Project</b> </center>


# Protecting Location Privacy with Geomasking
<i>Yue Lin</i>

This Jupyter notebook demonstrates how geomasking techniques can be used to protect the location privacy of individual-level sensitive data.